In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
# Сгенерируем колонки
def generate_columns(n, name):
    columns = [name+str(i) for i in range(1, n+1)]
    return columns

In [3]:
# Убираем лишние нолики, чтобы красивее было смотреть
def drop_zero(n, except_k):
    cols = set(generate_columns(n, "#host")) | set(generate_columns(n, "POShost"))
    drop = cols - {f"#host{except_k}", f"POShost{except_k}"}
    return list(drop)

In [4]:
# Смотрим на конструкции по нужным столбцам и слову
# Можно попросить показать прочерки
# Можно задать верхнюю и нижнюю границы отношения
# Можно дать дф, из которого надо смотреть, по умолчанию -- из файлов all_with_ratios_
def watch_constr(n, cols="all", host_of=None, bott_lim=0.0, up_lim=1.0, outhost=False, entries_lim=0, source="file"):
    if isinstance(source, pd.DataFrame):
        df = source
    else:
        df = pd.read_csv(f"./outcome files/dev_train_test/all_start_finish/all_with_ratios_{n}.csv", sep=",", low_memory=False)
    
    search_patt = ((df["ratio"] >= bott_lim) & 
                  (df["ratio"] <= up_lim) &
                  (df["total_entries"] >= entries_lim))
    if cols == "all":
        cols = ["any"] * n
    for i in range(n):
        if cols[i] != "any":
            search_patt &= (df[f"POS{i+1}"] == cols[i])
        if not outhost:
            search_patt &= (df[f"POShost{i+1}"] != '_')
    if host_of:
        search_patt &= (df[f"#host{host_of}"] != "0")
        return df[search_patt].drop(columns=drop_zero(n, host_of))
    else:
        return df[search_patt]

.




# Соединение

**План:**
1. Сохранить ВСЕ крутые конструкции (с отношением =>0.97) для n-грамм
2. Для оставшихся (пока вместе с остатками от крутых) пойти в n+1-граммы
3. Добавить к ним по слову с каждого конца (если появилась связь у добаленного, не трогаем, она потом вылезет)
4. Сохранить из получившихся n+1-грамм крутые

(про только популярные (>=20 вхождений) можно посчитать потом и сравнить с заранее преобработанными; в таком случае выбросим 966 вхождений при 3-граммах

UPD: оставила столбцы с количеством вхождений, чтобы потом можно было отбирать с нужным лимитом: последний столбец -- число только хороших, а предпоследний -- вместе с плохими)


**Вопросы:**

- в тройках пока никого не склеиваю, но есть места, где можно схлопнуть контекст
- если так объединять, то надо ли и тут записывать с минусами
- зачем нам минусы, что даст эта полная группа?(
- те, у кого нет с минусами, но не вылезли в 3-граммах -- это из-за связей-прочерков, т.е. снаружи.
- пока одна конструкция -- одно подчинение (а не все связи в конструкции), но мб имеет смысл схлопнуть в одн строчку тех, кто без списков

In [5]:
# Записывает в csv конструкцию + число её вхождений без лишних нулей
# n -- разме n-граммы
# cool -- датафрейм/список для записи
# mode -- режим (w -- запись с нуля, a -- добавление в существующий файл)
# all_host -- у всех ли слов есть инфа про хосты (при расширении контекста у добавленных слов нет заплаток-ноликов)
def write_cool(n, cool, mode, file_dir, all_host=True):
    if isinstance(cool, list):
        cool_w = np.array(cool)
    else:
        cool_w = cool.to_numpy()
    x, y = cool_w.shape
    if all_host:
        del_col = n - 1
    else:
        del_col = n - 2
    cool_w = cool_w[cool_w != "0"].reshape((x, y - del_col))
    
    with open(file_dir + f"/all_cool_{n}.csv", mode, newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in cool_w:
            writer.writerow(row)

In [6]:
# Удаляет из df его почти*_кусочек subdf
# *почти -- потому что колонки не совпадают, для этого мерджим
# Возвращает обновленный new_df
def delete_subdf(df, subdf):
    del_df = pd.merge(subdf, df, on=list(subdf.columns))
    print("Rows to delete:", del_df.shape[0])
    new_df = pd.concat([df, del_df]).drop_duplicates(keep=False)
    return new_df

In [7]:
# Передвигает названия колонок и номера связей
# (кажется, это проще, чем задавать для 4-грамм конструкции по условиям)
# n -- размер n-граммы
# df -- датафрейм, в котором нужно передвинуть колонки
# shift_by -- на сколько значений передвинуть, по умолчанию на 1
def shift_n(n, df, shift_by=1):
    df.columns = [col_name[:-1] + str(int(col_name[-1]) + 1) for col_name in list(df.columns)]
    for x in range (shift_by + 1, n + shift_by + 1):
        df[f"#host{x}"] = df[f"#host{x}"].apply(lambda x: str(int(x) + 1) if x != "0" else "0")
        
    return df

In [8]:
cool_3 = watch_constr(3, "all", bott_lim=0.97).drop(["#construction", "ratio"] + generate_columns(3, "POShost"), axis=1)
cool_3.columns = list(cool_3.columns)[:-1] + ["total_entries_all"]
cool_3["total_entries_good"] = cool_3["total_entries_all"]
write_cool(3, cool_3, mode="w")

In [9]:
# Из 4-грамм убираем уже записанные 3-граммы...
# ...с дополнительным словом справа...
all_4 = watch_constr(4, "all")
new_all_4 = delete_subdf(all_4, cool_3.drop(["total_entries_all", "total_entries_good"], axis=1)) # удалили с добавкой справа

Rows to delete: 1972


In [10]:
# ...и с дополнительным словом слева
cool_3_copy = cool_3.copy(deep=True).drop(["total_entries_all", "total_entries_good"], axis=1)
cool_3_l = shift_n(3, cool_3_copy)
    
new_all_4 = delete_subdf(new_all_4, cool_3_l)

Rows to delete: 1672


In [11]:
# ! Для записи с минусами везде ниже new_cool_4 > new_all_4
# new_cool_4 = watch_constr(4, "all", bott_lim=0.97, source=new_all_4)

In [12]:
# Не_очень 3-граммы
meh_3 = watch_constr(3, "all", up_lim=0.9699).drop(["#construction", "ratio", "total_entries"], axis=1)

### Добавляем контекст справа

In [8]:
# Засширяет до хороших n+1-грамм заданные плохие n-граммы и с правым, и с левым контекстом
# Возвращает датафрейм сжатых хороших для записи в файл и датафрейм развернутых хороших для удаления из общей массы n+1-грамм
# bigger_n -- размер n+1-граммы, то, до чего расширили
# meh_df -- датафрейм плохих конструкций, которые хотим улучшить (смещается вправо/влево отдельно)
# bigger_df -- датафрейм n+1-грамм с вычетом конструкций, уже обработанных на данный момент
# side -- right=слово добавляется справа, left=слева
def widening_by_one(bigger_n, meh_df, bigger_df, side):
    
    # Для интересующих нас конструкций расширяем конекст
    saved = pd.merge(bigger_df, meh_df, on=list(meh_df.columns)).drop(generate_columns(bigger_n, "POShost"), axis=1)
    
    # Набор и порядок колонок в зависимости от стороны контекста
    if side == "right":
        group_list = list(saved.columns[1:-4])
        new_pos = f"POS{bigger_n}"
        final_order = group_list + [f"{new_pos}_ok", f"{new_pos}_minus", "total_entries_all", "total_entries_good"]
    elif side == "left":
        group_list = list(saved.columns[3:-2])
        new_pos = "POS1"
        final_order = [f"{new_pos}_ok", f"{new_pos}_minus"] + group_list + ["total_entries_all", "total_entries_good"]
        
    # Слова, с которыми хорошо
    add_pos = saved[saved["ratio"] >= 0.97].groupby(group_list)[new_pos].apply(lambda pos: ','.join(pos))
    
    # Слова, с которыми плохо, заносим с "_"
    add_minus_pos = saved[saved["ratio"] < 0.9699].groupby(group_list)[new_pos].apply(lambda pos: ','.join("_" + pos))
    
    # Соединяем получившиеся колонки
    both_pos = pd.merge(add_pos, add_minus_pos, on=group_list,\
                        how="outer", suffixes=('_ok', '_minus'), sort=group_list)
    # Добавляем к ним сумму по всем вхождениям и только по хорошим
    sum_entries = saved.groupby(group_list)["total_entries"].sum()
    bigger_cool = pd.merge(both_pos, sum_entries, on=group_list).dropna(subset=[f"{new_pos}_ok"])
    sum_good_entries = saved[saved["ratio"] >= 0.97].groupby(group_list)["total_entries"].sum()
    bigger_cool = pd.merge(bigger_cool, sum_good_entries, on=group_list, suffixes=('_all', '_good'))
    
    # Избавляемся от мультииндекса и перестраиваем колонки в нужном порядке
    bigger_cool.reset_index(inplace=True)
    bigger_cool = bigger_cool[final_order]

    return bigger_cool, saved

In [14]:
wid_3_right, saved_3_right = widening_by_one(4, meh_3, new_all_4, side="right")
wid_3_right.head()

,POS1,#host1,POS2,#host2,POS3,#host3,POS4_ok,POS4_minus,total_entries_all,total_entries_good
0,ADJ,0,ADJ,0,ADV,2,"NOUN,PUNCT",NaN,3,3
1,ADJ,0,ADJ,0,AUX,2,"ADP,ADV,PART",_VERB,6,3
2,ADJ,0,ADJ,0,NUM,2,DET,NaN,1,1
3,ADJ,0,ADJ,0,PART,2,NOUN,NaN,1,1
4,ADJ,0,ADJ,0,VERB,2,"NUM,PRON","_ADJ,_ADP,_NOUN,_PUNCT",9,2


In [15]:
write_cool(4, wid_3_right, mode="w", all_host=False)

### Добавляем контекст слева

In [16]:
# Сдвигаем номера колонок
meh_3_copy = meh_3.copy(deep=True)
meh_3_l = shift_n(3, meh_3_copy)

In [17]:
# Удаляем уже отобранное, потому что оно в большинстве случаев (94 из 108) дублируется!
new_all_4 = delete_subdf(new_all_4, saved_3_right[saved_3_right["ratio"] >= 0.97].drop(["#construction", "ratio"], axis=1))

Rows to delete: 6500


In [18]:
wid_3_left, saved_3_left = widening_by_one(4, meh_3_l, new_all_4, side="left")
wid_3_left.head()

,POS1_ok,POS1_minus,POS2,#host2,POS3,#host3,POS4,#host4,total_entries_all,total_entries_good
0,"NOUN,SCONJ","_ADV,_CCONJ",ADJ,0,ADJ,0,AUX,3,6,3
1,ADV,NaN,ADJ,0,ADJ,0,PART,3,1,1
2,"DET,NUM,VERB","_CCONJ,_NOUN,_PRON",ADJ,0,ADJ,0,PRON,3,6,3
3,AUX,"_ADV,_CCONJ,_NOUN,_PRON",ADJ,0,ADJ,0,VERB,3,8,1
4,"ADJ,ADV,CCONJ,DET,PROPN,SCONJ","_ADP,_NOUN,_PART,_PRON,_PUNCT,_VERB",ADJ,0,ADJ,4,PROPN,0,60,19


In [19]:
write_cool(4, wid_3_left, mode="a", all_host=False)

### Дописываем оставшиеся хорошие 4-граммы

In [20]:
pure_cool_4 = delete_subdf(new_all_4, saved_3_left[saved_3_left["ratio"] >= 0.97].drop(["#construction", "ratio"], axis=1))
pure_cool_4 = pure_cool_4[pure_cool_4["ratio"] >= 0.97]
pure_cool_4.columns = list(pure_cool_4.columns)[:-1] + ["total_entries_all"]
pure_cool_4["total_entries_good"] = pure_cool_4["total_entries_all"]
write_cool(4, pure_cool_4.drop(["#construction", "ratio"] + generate_columns(4, "POShost"), axis=1), mode="a")

Rows to delete: 3781


In [21]:
pure_cool_4["#host4"].value_counts()

0    377
1    259
Name: #host4, dtype: int64

In [9]:
# Чтение готовых конструкций
def read_ready(csvfile, lim_all=0, lim_good=0):
    ready = []
    with open(csvfile, newline='') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            if int(row[-1]) >= lim_good and int(row[-2]) >= lim_all:
                line = []
                for word in row[:-2]:
                    if word == "nan":
                        pass
                    elif "," in word:
                        word_list = word.split(',')
                        line.append(word_list)
                    else:
                        line.append(word)
                ready.append(line)         
    return ready

In [10]:
read_ready("./outcome files/...../all_cool_4.csv")[:5]

[['ADJ', 'ADJ', 'ADV', '2', ['NOUN', 'PUNCT']],
 ['ADJ', 'ADJ', 'AUX', '2', ['ADP', 'ADV', 'PART'], '_VERB'],
 ['ADJ', 'ADJ', 'NUM', '2', 'DET'],
 ['ADJ', 'ADJ', 'PART', '2', 'NOUN'],
 ['ADJ',
  'ADJ',
  'VERB',
  '2',
  ['NUM', 'PRON'],
  ['_ADJ', '_ADP', '_NOUN', '_PUNCT']]]